In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import matplotlib.backends.backend_pdf

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

%matplotlib inline

dir_data='/Users/ahenny/'
dir1='/Volumes/LaCie/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
yrs=np.arange(1950,2017,1)
ratios_regen_land_list=[]
ratios_regen_ocean_list=[]

ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_1950-2016_QualityMask.nc')
quality=ds['p']

ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
t2m=ds['t2m'][0,:,:]
land_mask=t2m/t2m
land_mask=land_mask.fillna(0)
        
lon_east=[x for x in land_mask.longitude.values if x>=180.]
lon_west=[x for x in land_mask.longitude.values if x<180.]
l_east=land_mask.sel(longitude=lon_east)
l_west=land_mask.sel(longitude=lon_west)
l_east['longitude']=[x-360. for x in lon_east]
land_mask=xr.concat([l_east,l_west],dim='longitude')
        
dk=xr.Dataset()
dk['land']=(('lat','lon'),land_mask.values)
dk.coords['lat']=land_mask.latitude.values
dk.coords['lon']=land_mask.longitude.values
        
land_mask=dk['land'].fillna(0)


for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    if 1==1:
        ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_'+str(year)+'.nc')
        lats_sel=[x for x in ds.lat.values if -60<=x<=70]
        precip=ds['p'].sel(lat=lats_sel)#.where(quality==1)
        weights=np.cos(np.deg2rad(precip.lat))
        land_mask_interp=land_mask.interp_like(precip[0,:,:])
        
        defined=xr.ones_like(precip)
        defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp>=0.5).weighted(weights)
        #defined=defined.where(precip>=-0.01).fillna(0).where(quality==1).weighted(weights)
        defined=defined.mean().values.tolist()
        
        defined1=xr.ones_like(precip)
        defined1=defined1.fillna(0).where(land_mask_interp>=0.5).weighted(weights)
        defined1=defined1.mean().values.tolist()

        ratio=float(defined)/float(defined1)
        ratios_regen_land_list.append(ratio)
        
        defined=xr.ones_like(precip)
        defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp<=0.5).weighted(weights)
        #defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp<=0.5).where(quality==1).weighted(weights)
        defined=defined.mean().values.tolist()
        
        defined1=xr.ones_like(precip)
        #defined1=defined1.fillna(0).where(land_mask_interp<=0.5).where(quality==1).weighted(weights)
        defined1=defined1.fillna(0).where(land_mask_interp<=0.5).weighted(weights)
        defined1=defined1.mean().values.tolist()
        
        ratio=float(defined)/float(defined1)
        ratios_regen_ocean_list.append(ratio)

In [ ]:
yrs=np.arange(1982,2021,1)
ratios_gpcc_land_list=[]
ratios_gpcc_ocean_list=[]

ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
t2m=ds['t2m'][0,:,:]
land_mask=t2m/t2m
land_mask=land_mask.fillna(0)
        
lon_east=[x for x in land_mask.longitude.values if x>=180.]
lon_west=[x for x in land_mask.longitude.values if x<180.]
l_east=land_mask.sel(longitude=lon_east)
l_west=land_mask.sel(longitude=lon_west)
l_east['longitude']=[x-360. for x in lon_east]
land_mask=xr.concat([l_east,l_west],dim='longitude')
        
dk=xr.Dataset()
dk['land']=(('lat','lon'),land_mask.values)
dk.coords['lat']=land_mask.latitude.values
dk.coords['lon']=land_mask.longitude.values
        
land_mask=dk['land']


for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    if 1==1:
        ds=xr.open_dataset('/Users/ahenny/Downloads/full_data_daily_v2022_10_'+str(year)+'.nc')
        lats_sel=[x for x in ds.lat.values if -60<=x<=70]
        precip=ds['precip'].transpose('time','lat','lon').sel(lat=lats_sel)
        weights=np.cos(np.deg2rad(precip.lat))
        
        land_mask_interp=land_mask.interp_like(precip[0,:,:])
        
        defined=xr.ones_like(precip)
        defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp>=0.5).weighted(weights)
        defined=defined.mean().values.tolist()
        
        defined1=xr.ones_like(precip)
        defined1=defined1.fillna(0).where(land_mask_interp>=0.5).weighted(weights)
        defined1=defined1.mean().values.tolist()

        ratio=float(defined)/float(defined1)
        ratios_gpcc_land_list.append(ratio)
        
        defined=xr.ones_like(precip)
        defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp<=0.5).weighted(weights)
        defined=defined.mean().values.tolist()
        
        defined1=xr.ones_like(precip)
        defined1=defined1.fillna(0).where(land_mask_interp<=0.5).weighted(weights)
        defined1=defined1.mean().values.tolist()
        
        ratio=float(defined)/float(defined1)
        ratios_gpcc_ocean_list.append(ratio)

In [ ]:
yrs=np.arange(1979,2024,1)
#yrs=np.arange(1983,2003,1)
ratios_mswep_land_list=[]
ratios_persiann_land_list=[]
ratios_cpc_land_list=[]
ratios_imerg_land_list=[]
ratios_gpcp_land_list=[]
ratios_gpcp3_land_list=[]
ratios_mswep_ocean_list=[]
ratios_persiann_ocean_list=[]
ratios_cpc_ocean_list=[]
ratios_imerg_ocean_list=[]
ratios_gpcp_ocean_list=[]
ratios_gpcp3_ocean_list=[]

ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
t2m=ds['t2m'][0,:,:]
land_mask=t2m/t2m
land_mask=land_mask.fillna(0)
        
lon_east=[x for x in land_mask.longitude.values if x>=180.]
lon_west=[x for x in land_mask.longitude.values if x<180.]
l_east=land_mask.sel(longitude=lon_east)
l_west=land_mask.sel(longitude=lon_west)
l_east['longitude']=[x-360. for x in lon_east]
land_mask=xr.concat([l_east,l_west],dim='longitude')
        
dk=xr.Dataset()
dk['land']=(('lat','lon'),land_mask.values)
dk.coords['lat']=land_mask.latitude.values
dk.coords['lon']=land_mask.longitude.values
        
land_mask=dk['land']


for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    if 1983<=year<=2022:
        ds=xr.open_dataset(dir_data+'holding/persiann_cdr_'+str(year)+'.nc')
        lats_sel=[x for x in ds.lat.values if -60<=x<=60]
        precip=ds['precip'].sel(lat=lats_sel)
        weights=np.cos(np.deg2rad(precip.lat))
        
        lons_east=[x for x in precip.lon.values if x>=180.]
        lons_west=[x for x in precip.lon.values if x<180.]
        p_east=precip.sel(lon=lons_east)
        p_west=precip.sel(lon=lons_west)
        p_east['lon']=[x-360. for x in lons_east]
        precip=xr.concat([p_east,p_west],dim='lon')
        p_east.close()
        p_west.close()
        
        land_mask_interp=land_mask.interp_like(precip[0,:,:])
        
        defined=xr.ones_like(precip)
        defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp>=0.5).weighted(weights)
        defined=defined.mean().values.tolist()
        
        defined1=xr.ones_like(precip)
        defined1=defined1.fillna(0).where(land_mask_interp>=0.5).weighted(weights)
        defined1=defined1.mean().values.tolist()

        ratio=float(defined)/float(defined1)
        ratios_persiann_land_list.append(ratio)
        
        defined=xr.ones_like(precip)
        defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp<=0.5).weighted(weights)
        defined=defined.mean().values.tolist()
        
        defined1=xr.ones_like(precip)
        defined1=defined1.fillna(0).where(land_mask_interp<=0.5).weighted(weights)
        defined1=defined1.mean().values.tolist()
        
        ratio=float(defined)/float(defined1)
        ratios_persiann_ocean_list.append(ratio)
        
    if 1979<=year<=2020:
        ds=xr.open_dataset(dir2+'mswep_past_deres_'+str(year)+'.nc')
        lats_sel=[x for x in ds.lat.values if -60<=x<=70]
        precip=ds['precip'].sel(lat=lats_sel)
        weights=np.cos(np.deg2rad(precip.lat))
        land_mask_interp=land_mask.interp_like(precip[0,:,:])

        defined=xr.ones_like(precip)
        defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp>=0.5).weighted(weights)
        defined=defined.mean().values.tolist()
        
        defined1=xr.ones_like(precip)
        defined1=defined1.fillna(0).where(land_mask_interp>=0.5).weighted(weights)
        defined1=defined1.mean().values.tolist()

        ratio=float(defined)/float(defined1)
        ratios_mswep_land_list.append(ratio)
        
        
        defined=xr.ones_like(precip)
        defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp<=0.5).weighted(weights)
        defined=defined.mean().values.tolist()
        
        defined1=xr.ones_like(precip)
        defined1=defined1.fillna(0).where(land_mask_interp<=0.5).weighted(weights)
        defined1=defined1.mean().values.tolist()
        
        ratio=float(defined)/float(defined1)
        ratios_mswep_ocean_list.append(ratio)
        
    if 1979<=year<=2023:
        ds=xr.open_dataset(dir2+'precip.'+str(year)+'.nc')
        lats_sel=[x for x in ds.lat.values if -60<=x<=70]
        precip=ds['precip'].sel(lat=lats_sel)
        weights=np.cos(np.deg2rad(precip.lat))
        
        lons_east=[x for x in precip.lon.values if x>=180.]
        lons_west=[x for x in precip.lon.values if x<180.]
        p_east=precip.sel(lon=lons_east)
        p_west=precip.sel(lon=lons_west)
        p_east['lon']=[x-360. for x in lons_east]
        precip=xr.concat([p_east,p_west],dim='lon')
        p_east.close()
        p_west.close()
        
        land_mask_interp=land_mask.interp_like(precip[0,:,:])

        defined=xr.ones_like(precip)
        defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp>=0.5).weighted(weights)
        defined=defined.mean().values.tolist()
        
        defined1=xr.ones_like(precip)
        defined1=defined1.fillna(0).where(land_mask_interp>=0.5).weighted(weights)
        defined1=defined1.mean().values.tolist()

        ratio=float(defined)/float(defined1)
        ratios_cpc_land_list.append(ratio)
        
        
        defined=xr.ones_like(precip)
        defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp<=0.5).weighted(weights)
        defined=defined.mean().values.tolist()
        
        defined1=xr.ones_like(precip)
        defined1=defined1.fillna(0).where(land_mask_interp<=0.5).weighted(weights)
        defined1=defined1.mean().values.tolist()
        
        ratio=float(defined)/float(defined1)
        ratios_cpc_ocean_list.append(ratio)
        
    if 2001<=year<=2022:
        ds=xr.open_dataset(dir3+'day/imerg_combined_deres_'+str(year)+'.nc')
        lats_sel=[x for x in ds.lat.values if -60<=x<=70]
        precip=ds['precip'].sel(lat=lats_sel)
        weights=np.cos(np.deg2rad(precip.lat))
        land_mask_interp=land_mask.interp_like(precip[0,:,:])

        defined=xr.ones_like(precip)
        defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp>=0.5).weighted(weights)
        defined=defined.mean().values.tolist()
        
        defined1=xr.ones_like(precip)
        defined1=defined1.fillna(0).where(land_mask_interp>=0.5).weighted(weights)
        defined1=defined1.mean().values.tolist()

        ratio=float(defined)/float(defined1)
        ratios_imerg_land_list.append(ratio)
        
        defined=xr.ones_like(precip)
        defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp<=0.5).weighted(weights)
        defined=defined.mean().values.tolist()
        
        defined1=xr.ones_like(precip)
        defined1=defined1.fillna(0).where(land_mask_interp<=0.5).weighted(weights)
        defined1=defined1.mean().values.tolist()
        
        ratio=float(defined)/float(defined1)
        ratios_imerg_ocean_list.append(ratio)
        
    if 1997<=year<=2022:
        ds=xr.open_dataset(dir_data+'holding/'+'gpcp_precip_'+str(year)+'.nc')
        lats_sel=[x for x in ds.lat.values if -60<=x<=70]
        precip=ds['precip'].sel(lat=lats_sel)
        weights=np.cos(np.deg2rad(precip.lat))
        land_mask_interp=land_mask.interp_like(precip[0,:,:])
        
        defined=xr.ones_like(precip)
        defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp>=0.5).weighted(weights)
        defined=defined.mean().values.tolist()
        
        defined1=xr.ones_like(precip)
        defined1=defined1.fillna(0).where(land_mask_interp>=0.5).weighted(weights)
        defined1=defined1.mean().values.tolist()

        ratio=float(defined)/float(defined1)
        ratios_gpcp_land_list.append(ratio)
        
        defined=xr.ones_like(precip)
        defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp<=0.5).weighted(weights)
        defined=defined.mean().values.tolist()
        
        defined1=xr.ones_like(precip)
        defined1=defined1.fillna(0).where(land_mask_interp<=0.5).weighted(weights)
        defined1=defined1.mean().values.tolist()
        
        ratio=float(defined)/float(defined1)
        ratios_gpcp_ocean_list.append(ratio)
        
        
    if 2001<=year<=2020:
        ds=xr.open_dataset(dir_data+'holding/'+'gpcp3_precip_'+str(year)+'.nc')
        lats_sel=[x for x in ds.lat.values if -60<=x<=70]
        precip=ds['precip'].sel(lat=lats_sel)
        weights=np.cos(np.deg2rad(precip.lat))
        land_mask_interp=land_mask.interp_like(precip[0,:,:])
        
        defined=xr.ones_like(precip)
        defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp>=0.5).weighted(weights)
        defined=defined.mean().values.tolist()
        
        defined1=xr.ones_like(precip)
        defined1=defined1.fillna(0).where(land_mask_interp>=0.5).weighted(weights)
        defined1=defined1.mean().values.tolist()

        ratio=float(defined)/float(defined1)
        ratios_gpcp3_land_list.append(ratio)
        
        defined=xr.ones_like(precip)
        defined=defined.where(precip>=-0.01).fillna(0).where(land_mask_interp<=0.5).weighted(weights)
        defined=defined.mean().values.tolist()
        
        defined1=xr.ones_like(precip)
        defined1=defined1.fillna(0).where(land_mask_interp<=0.5).weighted(weights)
        defined1=defined1.mean().values.tolist()
        
        ratio=float(defined)/float(defined1)
        ratios_gpcp3_ocean_list.append(ratio)

In [ ]:
#ratios_cpc_list=[x/max(ratios_cpc_list) for x in ratios_cpc_list]

fig=plt.figure(figsize=(15,7))
ax=plt.subplot(1,1,1)
ax.plot(np.arange(1979,2024),ratios_cpc_land_list,color='limegreen',linewidth=5,label='CPC Unified')
ax.plot(np.arange(1979,2021),ratios_mswep_land_list,color='b',linewidth=5,label='MSWEP V2.8')
ax.plot(np.arange(1983,2023),ratios_persiann_land_list,color='orange',linewidth=5,label='PERSIANN-CDR')
ax.plot(np.arange(2001,2023),ratios_imerg_land_list,color='magenta',linewidth=5,label='IMERG V07a')
ax.plot(np.arange(1997,2023),ratios_gpcp_land_list,color='cyan',linewidth=5,label='GPCP Daily CDR')
ax.plot(np.arange(2001,2021),ratios_gpcp3_land_list,color='gold',linewidth=5,label='GPCP V3.2')
ax.plot(np.arange(1950,2017),ratios_regen_land_list,color='k',linewidth=5,label='REGEN')
ax.plot(np.arange(1982,2021),ratios_gpcc_land_list,color='darkviolet',linewidth=5,label='GPCC V2022')
ax.tick_params(labelsize=20)
ax.set_title(r'$\bf{60S-70N}$'+' '+r'$\bf{Land}$',fontsize=30,pad=5)
ax.set_xlabel('Year',fontsize=22)
ax.set_ylabel('Fraction of area present',fontsize=22)
ax.grid(True)
plt.legend(loc='lower left',fontsize=17)
#plt.yticks([0.85,0.9,0.95,1])
#ax.set_ylim(0.805,1.01)
ax.set_xlim(1947,2024)
ax.set_ylim(0.47,1.02)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.5)
plt.show()

In [ ]:
fig.savefig(dir_data+'completeness_lines_1.png')

In [ ]:
#ratios_cpc_list=[x/max(ratios_cpc_list) for x in ratios_cpc_list]

fig=plt.figure(figsize=(15,7))
ax=plt.subplot(1,1,1)
#ax.plot(np.arange(1979,2024),ratios_cpc_ocean_list,color='limegreen',linewidth=5,label='CPC Unified')
ax.plot(np.arange(1979,2021),ratios_mswep_ocean_list,color='b',linewidth=5,label='MSWEP V2.8')
ax.plot(np.arange(1983,2023),ratios_persiann_ocean_list,color='orange',linewidth=5,label='PERSIANN-CDR')
ax.plot(np.arange(2001,2023),ratios_imerg_ocean_list,color='magenta',linewidth=5,label='IMERG V07a')
ax.plot(np.arange(1997,2023),ratios_gpcp_ocean_list,color='cyan',linewidth=5,label='GPCP Daily CDR')
ax.plot(np.arange(2001,2021),ratios_gpcp3_ocean_list,color='gold',linewidth=5,label='GPCP V3.2')
#ax.plot(np.arange(1950,2017),ratios_regen_ocean_list,color='k',linewidth=5,label='REGEN')
#ax.plot(np.arange(1982,2021),ratios_gpcc_ocean_list,color='darkviolet',linewidth=5,label='GPCC V2022')
ax.tick_params(labelsize=20)
ax.set_title(r'$\bf{60S-70N}$'+' '+r'$\bf{Ocean}$',fontsize=30,pad=5)
ax.set_xlabel('Year',fontsize=22)
ax.set_ylabel('Fraction of area present',fontsize=22)
ax.grid(True)
plt.legend(loc='lower left',fontsize=17)
#plt.yticks([0.85,0.9,0.95,1])
#ax.set_ylim(0.805,1.01)
ax.set_xlim(1947,2024)
ax.set_ylim(0.47,1.02)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.5)
plt.show()

In [ ]:
fig.savefig(dir_data+'completeness_lines_2.png')

In [ ]:
yrs=np.arange(1983,2023,1)
#yrs=np.arange(1983,2003,1)

total_length=0
for i in range(len(yrs)):
    year=yrs[i]
    print(year)

    ds=xr.open_dataset(dir_data+'holding/persiann_cdr_'+str(year)+'.nc')
    precip=ds['precip']
    
    if i==0:
        defined_sum=xr.zeros_like(precip[0,:,:])

    defined=xr.ones_like(precip)
    defined=defined.where(precip>=-0.01)
    defined=defined.sum(dim='time',skipna=True)
    defined_sum=defined_sum+defined
    total_length=total_length+precip.time.size
    
defined_ratio=defined_sum/total_length

In [ ]:
#fig=plt.figure(figsize=(20,9.5))
fig=plt.figure(figsize=(21,6.5))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

#cax=ax.pcolormesh(defined_sum.lon.values,defined_sum.lat.values,defined_sum/15340.*100.,vmin=80,vmax=100,cmap=plt.cm.viridis)
cax=ax.pcolormesh(defined_sum.lon.values,defined_sum.lat.values,defined_ratio*100,vmin=90,vmax=100,cmap=plt.cm.viridis)

cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,extend='min')
cbar.ax.tick_params(labelsize=24) 
#cbar.set_label('days yr$^{-1}$',fontsize=25,labelpad=5)
cbar.set_label('% grids present',fontsize=25,labelpad=5)
ax.coastlines(resolution='10m')
#ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
#ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
ax.set_title(r'$\bf{PERSIANN-CDR}$'+' '+r'$\bf{(1983-2022)}$',fontsize=34,pad=5)
ax.set_ylim(-60,70)
plt.show()

In [ ]:
fig.savefig(dir_data+'completeness_geoplots_1.png')

In [ ]:
yrs=np.arange(1979,2023,1)
#yrs=np.arange(1983,2003,1)

total_length=0
for i in range(len(yrs)):
    year=yrs[i]
    print(year)

    ds=xr.open_dataset(dir_data+'precip.'+str(year)+'.nc')
    precip=ds['precip']#already -89.75 to 89.75
        
    lons_east=[x for x in precip.lon.values if x>=180.]
    lons_west=[x for x in precip.lon.values if x<180.]
    p_east=precip.sel(lon=lons_east)
    p_west=precip.sel(lon=lons_west)
    p_east['lon']=[x-360. for x in lons_east]
    precip=xr.concat([p_east,p_west],dim='lon')
    p_east.close()
    p_west.close()
    
    if i==0:
        defined_sum=xr.zeros_like(precip[0,:,:])

    defined=xr.ones_like(precip)
    defined=defined.where(precip>=-0.01)
    defined=defined.sum(dim='time',skipna=True)
    defined_sum=defined_sum+defined
    total_length=total_length+precip.time.size
    
defined_ratio=defined_sum/total_length

In [ ]:
fig=plt.figure(figsize=(21,6.5))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

cax=ax.pcolormesh(defined_sum.lon.values,defined_sum.lat.values,defined_ratio*100,vmin=90,vmax=100,cmap=plt.cm.viridis)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,extend='min')
cbar.ax.tick_params(labelsize=24) 
cbar.set_label('% grids present',fontsize=25,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)

ax.set_ylim(-60,70)
ax.set_title(r'$\bf{CPC}$'+' '+r'$\bf{Unified}$'+' 'r'$\bf{(1979-2022)}$',fontsize=34,pad=5)
plt.show()

In [ ]:
fig.savefig(dir_data+'completeness_geoplots_2.png')

In [ ]:
yrs=np.arange(2001,2023,1)
#yrs=np.arange(1983,2003,1)

total_length=0
for i in range(len(yrs)):
    year=yrs[i]
    print(year)

    ds=xr.open_dataset(dir3+'day/imerg_combined_deres_'+str(year)+'.nc')
    precip=ds['precip']
    
    if i==0:
        defined_sum=xr.zeros_like(precip[0,:,:])

    defined=xr.ones_like(precip)
    defined=defined.where(precip>=-0.01)
    defined=defined.sum(dim='time',skipna=True)
    defined_sum=defined_sum+defined
    total_length=total_length+precip.time.size
    
defined_ratio=defined_sum/total_length

In [ ]:
fig=plt.figure(figsize=(21,6.5))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

cax=ax.pcolormesh(defined_sum.lon.values,defined_sum.lat.values,defined_ratio*100,vmin=90,vmax=100,cmap=plt.cm.viridis)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,extend='min')
cbar.ax.tick_params(labelsize=24) 
cbar.set_label('% grids present',fontsize=25,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)

ax.set_ylim(-60,70)
ax.set_title(r'$\bf{IMERG}$'+' '+r'$\bf{V07a}$'+' 'r'$\bf{(2001-2022)}$',fontsize=34,pad=5)
plt.show()

In [ ]:
fig.savefig(dir_data+'completeness_geoplots_3.png')

In [ ]:
yrs=np.arange(1979,2021,1)
#yrs=np.arange(1983,2003,1)

total_length=0
for i in range(len(yrs)):
    year=yrs[i]
    print(year)

    ds=xr.open_dataset(dir2+'mswep_past_deres_'+str(year)+'.nc')
    precip=ds['precip']
    
    if i==0:
        defined_sum=xr.zeros_like(precip[0,:,:])

    defined=xr.ones_like(precip)
    defined=defined.where(precip>=-0.01)
    defined=defined.sum(dim='time',skipna=True)
    defined_sum=defined_sum+defined
    total_length=total_length+precip.time.size
    
defined_ratio=defined_sum/total_length

In [ ]:
fig=plt.figure(figsize=(21,6.5))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

cax=ax.pcolormesh(defined_sum.lon.values,defined_sum.lat.values,defined_ratio*100,vmin=90,vmax=100,cmap=plt.cm.viridis)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,extend='min')
cbar.ax.tick_params(labelsize=24) 
cbar.set_label('% grids present',fontsize=25,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)

ax.set_ylim(-60,70)
ax.set_title(r'$\bf{MSWEP}$'+' '+r'$\bf{V2.8}$'+' 'r'$\bf{(1979-2020)}$',fontsize=34,pad=5)
plt.show()

In [ ]:
fig.savefig(dir_data+'completeness_geoplots_4.png')

In [ ]:
yrs=np.arange(1997,2023,1)
#yrs=np.arange(1983,2003,1)

total_length=0
for i in range(len(yrs)):
    year=yrs[i]
    print(year)

    ds=xr.open_dataset(dir_data+'holding/'+'gpcp_precip_'+str(year)+'.nc')
    precip=ds['precip']
    
    if i==0:
        defined_sum=xr.zeros_like(precip[0,:,:])

    defined=xr.ones_like(precip)
    defined=defined.where(precip>=-0.01)
    defined=defined.sum(dim='time',skipna=True)
    defined_sum=defined_sum+defined
    total_length=total_length+precip.time.size
    
defined_ratio=defined_sum/total_length

In [ ]:
fig=plt.figure(figsize=(21,6.5))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

cax=ax.pcolormesh(defined_sum.lon.values,defined_sum.lat.values,defined_ratio*100,vmin=90,vmax=100,cmap=plt.cm.viridis)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,extend='min')
cbar.ax.tick_params(labelsize=24) 
cbar.set_label('% grids present',fontsize=25,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)

ax.set_ylim(-60,70)
ax.set_title(r'$\bf{GPCP}$'+' '+r'$\bf{Daily}$'+' '+r'$\bf{CDR}$'+' 'r'$\bf{(1997-2022)}$',fontsize=34,pad=5)
plt.show()

In [ ]:
fig.savefig(dir_data+'completeness_geoplots_5.png')

In [ ]:
yrs=np.arange(2001,2021,1)

total_length=0
for i in range(len(yrs)):
    year=yrs[i]
    print(year)

    ds=xr.open_dataset(dir_data+'holding/'+'gpcp3_precip_'+str(year)+'.nc')
    precip=ds['precip']
    
    if i==0:
        defined_sum=xr.zeros_like(precip[0,:,:])

    defined=xr.ones_like(precip)
    defined=defined.where(precip>=-0.01)
    defined=defined.sum(dim='time',skipna=True)
    defined_sum=defined_sum+defined
    total_length=total_length+precip.time.size
    
defined_ratio=defined_sum/total_length

In [ ]:
fig=plt.figure(figsize=(21,6.5))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

cax=ax.pcolormesh(defined_sum.lon.values,defined_sum.lat.values,defined_ratio*100,vmin=90,vmax=100,cmap=plt.cm.viridis)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,extend='min')
cbar.ax.tick_params(labelsize=24) 
cbar.set_label('% grids present',fontsize=25,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)

ax.set_ylim(-60,70)
ax.set_title(r'$\bf{GPCP}$'+' '+r'$\bf{V3.2}$'+' 'r'$\bf{(2001-2020)}$',fontsize=34,pad=5)
plt.show()

In [ ]:
fig.savefig(dir_data+'completeness_geoplots_8.png')

In [ ]:
yrs=np.arange(1950,2017,1)
#yrs=np.arange(1983,2003,1)

total_length=0
for i in range(len(yrs)):
    year=yrs[i]
    print(year)

    ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_'+str(year)+'.nc')
    precip=ds['p']#.where(quality==1)
    
    if i==0:
        defined_sum=xr.zeros_like(precip[0,:,:])

    defined=xr.ones_like(precip)
    defined=defined.where(precip>=-0.01)
    defined=defined.sum(dim='time',skipna=True)
    defined_sum=defined_sum+defined
    total_length=total_length+precip.time.size
    
defined_ratio=defined_sum/total_length

In [ ]:
fig=plt.figure(figsize=(21,6.5))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

cax=ax.pcolormesh(defined_sum.lon.values,defined_sum.lat.values,defined_ratio*100,vmin=90,vmax=100,cmap=plt.cm.viridis)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,extend='min')
cbar.ax.tick_params(labelsize=24) 
cbar.set_label('% grids present',fontsize=25,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)

ax.set_ylim(-60,70)
#ax.set_title(r'$\bf{REGEN}$'+' '+r'$\bf{HQ}$'+' 'r'$\bf{(1950-2016)}$',fontsize=34,pad=5)
ax.set_title(r'$\bf{REGEN}$'+' 'r'$\bf{(1950-2016)}$',fontsize=34,pad=5)
plt.show()

In [ ]:
fig.savefig(dir_data+'completeness_geoplots_6a.png')

In [ ]:
yrs=np.arange(1982,2021,1)
#yrs=np.arange(1983,2003,1)

total_length=0
for i in range(len(yrs)):
    year=yrs[i]
    print(year)

    ds=xr.open_dataset('/Users/ahenny/Downloads/full_data_daily_v2022_10_'+str(year)+'.nc')
    precip=ds['precip'].transpose('time','lat','lon')
    
    if i==0:
        defined_sum=xr.zeros_like(precip[0,:,:])

    defined=xr.ones_like(precip)
    defined=defined.where(precip>=-0.01)
    defined=defined.sum(dim='time',skipna=True)
    defined_sum=defined_sum+defined
    total_length=total_length+precip.time.size
    
defined_ratio=defined_sum/total_length

In [ ]:
fig=plt.figure(figsize=(21,6.5))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

cax=ax.pcolormesh(defined_sum.lon.values,defined_sum.lat.values,defined_ratio*100,vmin=90,vmax=100,cmap=plt.cm.viridis)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,extend='min')
cbar.ax.tick_params(labelsize=24) 
cbar.set_label('% grids present',fontsize=25,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)

ax.set_ylim(-60,70)
ax.set_title(r'$\bf{GPCC}$'+' '+r'$\bf{V2022}$'+' 'r'$\bf{(1982-2020)}$',fontsize=34,pad=5)
plt.show()

In [ ]:
fig.savefig(dir_data+'completeness_geoplots_7.png')

In [ ]:
fig=plt.figure(figsize=(20,9.5))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.pcolormesh(defined_sum.lon.values,defined_sum.lat.values,(defined_ratio1-defined_ratio)*100,vmin=0,vmax=100,cmap=plt.cm.viridis)

cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=35,extend='both')
cbar.ax.tick_params(labelsize=24) 
cbar.set_label('% difference',fontsize=25,labelpad=5)
ax.coastlines(resolution='10m',color='w')
#countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
#ax.add_feature(countries)

ax.set_title('CPC completeness change',fontsize=40,pad=5)
plt.show()

In [ ]:
fig.savefig(dir_data+'cpc_completeness_1.png')